# Улучшение качества  POS-taggera

ЗАДАНИЕ

на вебинаре был рассмотрен пример тегирования с использованием корпуса на Русском языке, вам необходимо улучшить модель что бы качество классификации было выше чем с лог регрессией.

Т.к библиотека deeppavlov у меня не может запуститься использую логистическую регрессию и улучшаю ее используя гиперпараметры.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [2]:
!pip install pyconll

In [3]:
import pyconll

In [4]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2020-12-06 17:31:32--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81043533 (77M) [text/plain]
Saving to: './datasets/ru_syntagrus-ud-train.conllu'

     0K .......... .......... .......... .......... ..........  0%  223K 5m55s
    50K .......... .......... .......... .......... ..........  0%  481K 4m20s
   100K .......... .......... .......... .......... ..........  0%  291K 4m23s
   150K .......... .......... .......... .......... ..........  0% 1,73M 3m29s
   200K .......... .......... .......... .......... ..........  0% 2,75M 2m52s
   250K .......... .......... .......... .......... ..........  0% 1016K 2m36s
   300K .......... ........

 46450K .......... .......... .......... .......... .......... 58% 1,68M 16s
 46500K .......... .......... .......... .......... .......... 58% 1,31M 16s
 46550K .......... .......... .......... .......... .......... 58% 1,66M 16s
 46600K .......... .......... .......... .......... .......... 58% 3,22M 16s
 46650K .......... .......... .......... .......... .......... 59% 2,43M 16s
 46700K .......... .......... .......... .......... .......... 59% 3,15M 16s
 46750K .......... .......... .......... .......... .......... 59% 1,40M 16s
 46800K .......... .......... .......... .......... .......... 59% 3,03M 16s
 46850K .......... .......... .......... .......... .......... 59% 1,32M 16s
 46900K .......... .......... .......... .......... .......... 59% 1,48M 16s
 46950K .......... .......... .......... .......... .......... 59% 2,64M 16s
 47000K .......... .......... .......... .......... .......... 59% 2,06M 16s
 47050K .......... .......... .......... .......... .......... 59% 1,78M 16s

 69950K .......... .......... .......... .......... .......... 88% 1,18M 5s
 70000K .......... .......... .......... .......... .......... 88% 2,14M 5s
 70050K .......... .......... .......... .......... .......... 88% 1,09M 5s
 70100K .......... .......... .......... .......... .......... 88% 3,50M 4s
 70150K .......... .......... .......... .......... .......... 88% 2,23M 4s
 70200K .......... .......... .......... .......... .......... 88% 1,62M 4s
 70250K .......... .......... .......... .......... .......... 88% 2,39M 4s
 70300K .......... .......... .......... .......... .......... 88%  516K 4s
 70350K .......... .......... .......... .......... .......... 88% 14,3M 4s
 70400K .......... .......... .......... .......... .......... 89% 11,1M 4s
 70450K .......... .......... .......... .......... .......... 89% 9,53M 4s
 70500K .......... .......... .......... .......... .......... 89% 16,3M 4s
 70550K .......... .......... .......... .......... .......... 89% 1,86M 4s
 70600K ....

--2020-12-06 17:32:15--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10903424 (10M) [text/plain]
Saving to: './datasets/ru_syntagrus-ud-dev.conllu'

     0K .......... .......... .......... .......... ..........  0%  124K 85s
    50K .......... .......... .......... .......... ..........  0%  557K 52s
   100K .......... .......... .......... .......... ..........  1%  754K 39s
   150K .......... .......... .......... .......... ..........  1% 1,62M 31s
   200K .......... .......... .......... .......... ..........  2% 1,66M 26s
   250K .......... .......... .......... .......... ..........  2% 1,66M 22s
   300K .......... .......... .......... .......... ..........  3% 2,14M 20s
   350

In [5]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')


  6950K .......... .......... .......... .......... .......... 65% 1,64M 2s
  7000K .......... .......... .......... .......... .......... 66% 1,85M 2s
  7050K .......... .......... .......... .......... .......... 66% 2,68M 2s
  7100K .......... .......... .......... .......... .......... 67% 1,88M 2s
  7150K .......... .......... .......... .......... .......... 67% 2,30M 2s
  7200K .......... .......... .......... .......... .......... 68% 1,67M 2s
  7250K .......... .......... .......... .......... .......... 68% 2,34M 2s
  7300K .......... .......... .......... .......... .......... 69% 2,07M 2s
  7350K .......... .......... .......... .......... .......... 69% 2,86M 2s
  7400K .......... .......... .......... .......... .......... 69% 1,86M 2s
  7450K .......... .......... .......... .......... .......... 70% 2,45M 2s
  7500K .......... .......... .......... .......... .......... 70% 2,13M 2s
  7550K .......... .......... .......... .......... .......... 71% 2,83M 2s
  7600K ...

In [6]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [7]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [8]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [9]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=200)

In [10]:
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [11]:
lr = LogisticRegression(random_state=0, max_iter=50)
lr.fit(X_train, train_enc_labels)

C:\Users\Windows\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
C:\Users\Windows\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\logistic.py:467: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  warnings.warn("Default multi_class will be changed to 'auto' in"


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.7843409833855693

Вывод - т.о. по сравнению с моделью лог. регрессии на вебинаре у нас точность возросла более чем на 15 %

# Создание признакового пространства

Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.

In [13]:
import pickle
with open('combine_df.pickle', 'rb') as f:
    combine_df = pickle.load(f)
    
combine_df.head()

,index,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized,tweet_vecs
0,0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy...","[[0.40891476115211844, 0.18360024644061923, -0..."
1,1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee...","[[0.4508066247491276, 0.17692823379355319, -0...."
2,2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]","[[0.7542365392049154, 0.5224650899569193, 0.00..."
3,3,4,0.0,#model i love u take with u all the time in ...,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[[0.49165773598684204, 0.2370210364460945, -0...."
4,4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]","[[0.3752712719142437, 0.1220138180651702, -0.1..."


ЗАДАНИЕ 1.

Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Повторим шаги из заданий 1 и 2, используя библиотеку nltk.

In [14]:
!pip install -U spacy

Requirement already up-to-date: spacy in c:\users\windows\anaconda3\envs\python38\lib\site-packages (2.3.4)


In [15]:
!pip install spacy download en

In [16]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    2.3.4                         
Location         C:\Users\Windows\Anaconda3\envs\python38\lib\site-packages\spacy
Platform         Windows-10-10.0.18362-SP0     
Python version   3.8.5                         
Models           en                            



In [17]:
!python -m spacy download en

символическая ссылка создана для C:\Users\Windows\Anaconda3\envs\python38\lib\site-packages\spacy\data\en <<===>> C:\Users\Windows\Anaconda3\envs\python38\lib\site-packages\en_core_web_sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[+] Linking successful
C:\Users\Windows\Anaconda3\envs\python38\lib\site-packages\en_core_web_sm -->
C:\Users\Windows\Anaconda3\envs\python38\lib\site-packages\spacy\data\en
You can now load the model via spacy.load('en')


In [18]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [19]:
!python -m spacy download en_core_web_md
!python -m spacy link en_core_web_md en --force

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
символическая ссылка создана для C:\Users\Windows\Anaconda3\envs\python38\lib\site-packages\spacy\data\en <<===>> C:\Users\Windows\Anaconda3\envs\python38\lib\site-packages\en_core_web_md
[+] Linking successful
C:\Users\Windows\Anaconda3\envs\python38\lib\site-packages\en_core_web_md -->
C:\Users\Windows\Anaconda3\envs\python38\lib\site-packages\spacy\data\en
You can now load the model via spacy.load('en')


In [20]:
import requests
from bs4 import BeautifulSoup
import re
import html5lib

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser') #'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [21]:
import spacy
from spacy import displacy
import en_core_web_md

nlp = en_core_web_md.load()


In [142]:
from collections import defaultdict, Counter
DELIMITER=';'

def get_popular_NER_spacy(text,popular_types=['ORG','GPE','PERSON'],top_count=20):
    pos_counts = defaultdict(Counter)
    doc = nlp(text)

    for token in doc:
        label=token.ent_type_
        if token.ent_iob==2 or token.ent_iob==0:
            label='UNKNOWN'            
        pos_counts[label + DELIMITER + str(token.orth)][token.orth] += 1
      
    sorted_pos_counts=sorted(pos_counts.items(),key=lambda item: max(list(item[1].values())),reverse=True)
    #print(sorted_pos_counts)
    
    i=0
    for label_pos, counts in sorted_pos_counts:        
        NER_type = label_pos[:label_pos.index(DELIMITER)]        
        if len(popular_types) > 0:
            if NER_type in popular_types:
                for orth_id, count in counts.most_common():
                    if i<top_count:
                        print(NER_type, count, doc.vocab.strings[orth_id])
                        i+=1
        else:
             for orth_id, count in counts.most_common():
                    if i<top_count:
                        print(NER_type, count, doc.vocab.strings[orth_id])
                        i+=1
            
    

In [117]:
combine_df['clean_tweet']

0        when father is dysfunctional and is so selfish...
1        thanks for lyft credit cannot use cause they d...
2                                      bihday your majesty
3                  model love take with all the time in ur
4                        factsguide society now motivation
                               ...                        
49154    thought factory left right polarisation trump ...
49155    feeling like mermaid hairflip neverready forma...
49156    hillary campaigned today in ohio omg used word...
49157    happy at work conference right mindset leads t...
49158    my song so glad free download shoegaze newmusi...
Name: clean_tweet, Length: 49159, dtype: object

In [125]:
txt=combine_df.to_string(columns=['clean_tweet'],header=False, index=False,index_names=False)


In [126]:
nlp.max_length=2507110 

In [143]:
get_popular_NER_spacy(txt)

PERSON 263 bihday
GPE 228 orlando
PERSON 182 bong
ORG 174 buffalo
PERSON 154 bing
GPE 133 america
ORG 107 ...
PERSON 102 obama
PERSON 102 sta
PERSON 102 bull
ORG 96 simulation
GPE 92 london
PERSON 83 th
PERSON 70 trump
PERSON 67 li
GPE 64 uk
ORG 63 the
PERSON 61 ...
ORG 58 bihday
GPE 57 st


Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным - самыми популярными типами являются ORG buffalo - 174 упоминаний,GPE orlando-228 упоминаний и PERSON bihday - 263 упоминаний


С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Из реальных персон вошли Обама и Трамп, организаций - buffalo, остальное вероятно мусор.

ЗАДАНИЕ 2.

Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?


In [146]:
import nltk

In [176]:
DELIMITER=';'
def get_popular_NER_nltk(text,popular_types=['ORG','GPE','PERSON'],top_count=20):   
    pos_counts = defaultdict(Counter)
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))):
        if hasattr(chunk, 'label'):
            word=' '.join(c[0] for c in chunk)
            pos_counts[chunk.label() + DELIMITER + word][word] += 1
        
    sorted_pos_counts=sorted(pos_counts.items(),key=lambda item: max(list(item[1].values())),reverse=True)
    #print(sorted_pos_counts)
    
    i=0
    for label_word, counts in sorted_pos_counts:        
        NER_type = label_word[:label_word.index(DELIMITER)]        
        if len(popular_types) > 0:
            if NER_type in popular_types:
                for word, count in counts.most_common():
                    if i<top_count:
                        print(NER_type, count, word)
                        i+=1
        else:
             for word, count in counts.most_common():
                    if i<top_count:
                        print(NER_type, count, word)
                        i+=1
   

In [177]:
get_popular_NER_nltk('In China thought factory left right polarisation Trump and in China we can see it.')

GPE 2 China
PERSON 1 Trump


Т.к сущности 'ORG','GPE','PERSON' должны начинаться с заглавных букв, то их get_popular_NER_nltk() не находит.
Заменим некоторые имена, организации и страны на заглавные.

In [180]:
txt=txt.replace('obama','Obama').replace('trump','Trump').replace('america','America').replace('orlando','Orlando').replace('buffalo','Buffalo')

In [181]:
get_popular_NER_nltk(txt)

PERSON 277 Trump
PERSON 193 Buffalo
GPE 142 America
PERSON 118 Orlando
PERSON 75 Obama
GPE 71 Orlando
GPE 53 American
GPE 13 Trump
GPE 7 Obama
PERSON 6 Trumps
GPE 4 Americans
GPE 4 Buffalo
GPE 4 Trumps
PERSON 3 Orlando Orlando
PERSON 3 Trump Obama
PERSON 3 Trump America
GPE 3 Americas
PERSON 2 America
PERSON 2 Obama Trump
GPE 2 Americanfootball


Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным - PERSON Trump - 277 упомининий, GPE  America -142 упоминаний и организаций найдено не было.

Какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор. Из реальных персон вошли Трамп и Обама, организации - не одной. Очень много мусора.

ЗАДАНИЕ 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.


Лучше отработала библиотека Spacy, там распознаются сущности не зависимо от регистра и меньше мусора. Кроме того, Spacy распознала Обаму и Трампа только как персоны, а Америку - только как GPE, что правильно, а nltk - и в персоны и в GPE было внесено много мусора, где и Обама и Трамп распознавались и как персоны и как GPE, а Америка - и как GPE и как персона.
Качество полученных most_common NER и количество распознаных NE безусловно лучше у Spacy чем у nltk.
